In [1]:
!pip install openpyxl

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import networkx as nx

In [3]:
data = pd.ExcelFile("../input/supply-chain-data/Supply chain logisitcs problem.xlsx")

In [4]:
plant_ports = data.parse("PlantPorts")
order_list = data.parse("OrderList")
products_per_plant = data.parse('ProductsPerPlant')
vmi_plants = data.parse("VmiCustomers")
freight_rates = data.parse("FreightRates")
wh_costs = data.parse("WhCosts")
wh_capacities = data.parse("WhCapacities")